In [4]:
!pip install langchain-core langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.1 MB/s eta 0:00:00


In [5]:
from langchain_core.tools import tool
from google.colab import userdata
# Retrieve the API key from Colab's secrets
api_key = userdata.get('openai_api_key')

In [2]:
@tool
def get_leave_policy() -> str:
    """Returns the company leave policy."""
    return "Employees are entitled to 20 paid leave days and 10 casual leave days per year."

@tool
def get_work_hours() -> str:
    """Returns the company working hours policy."""
    return "Regular working hours are from 9 AM to 6 PM, Monday to Friday."

@tool
def get_remote_policy() -> str:
    """Returns the remote work policy."""
    return "Employees can work remotely up to 3 days a week. Full remote requires approval."


In [6]:
from langchain_openai import ChatOpenAI

In [7]:
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=api_key)

In [8]:
# Bind all three tools
llm_with_tools = llm.bind_tools([
    get_leave_policy,
    get_work_hours,
    get_remote_policy
])

In [17]:
from langchain_core.messages import HumanMessage, ToolMessage

In [36]:
messages = [
    HumanMessage(
        "How many days work from home allowed and which are the timings?"
    )
]
llm_output = llm_with_tools.invoke(messages)
messages.append(llm_output)

In [38]:
messages

[HumanMessage(content='How many days work from home allowed and which are the timings?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_rAiSbFDac4Navby1sVsKSd3X', 'function': {'arguments': '{}', 'name': 'get_remote_policy'}, 'type': 'function'}, {'id': 'call_LPLixR4UMRKKfViXcjkwJG3M', 'function': {'arguments': '{}', 'name': 'get_work_hours'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 82, 'total_tokens': 124, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b376dfbbd5', 'id': 'chatcmpl-BJmSwBEZ2d7Th2y4hFbnA88GnTLEH', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-83eb0ef9-8645-45a4-a38c-f414238adcac-0', tool_calls=[{

In [39]:
tool_mapping = {
    "get_leave_policy": get_leave_policy,
    "get_work_hours": get_work_hours,
    "get_remote_policy": get_remote_policy,
}

In [40]:
llm_output.tool_calls

[{'name': 'get_remote_policy',
  'args': {},
  'id': 'call_rAiSbFDac4Navby1sVsKSd3X',
  'type': 'tool_call'},
 {'name': 'get_work_hours',
  'args': {},
  'id': 'call_LPLixR4UMRKKfViXcjkwJG3M',
  'type': 'tool_call'}]

In [41]:
for tool_call in llm_output.tool_calls:
    tool = tool_mapping[tool_call["name"].lower()]
    tool_output = tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

In [42]:
messages

[HumanMessage(content='How many days work from home allowed and which are the timings?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_rAiSbFDac4Navby1sVsKSd3X', 'function': {'arguments': '{}', 'name': 'get_remote_policy'}, 'type': 'function'}, {'id': 'call_LPLixR4UMRKKfViXcjkwJG3M', 'function': {'arguments': '{}', 'name': 'get_work_hours'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 82, 'total_tokens': 124, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b376dfbbd5', 'id': 'chatcmpl-BJmSwBEZ2d7Th2y4hFbnA88GnTLEH', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-83eb0ef9-8645-45a4-a38c-f414238adcac-0', tool_calls=[{

In [43]:
response=llm_with_tools.invoke(messages)
print(response.content)

Employees are allowed to work from home up to 3 days a week, and full remote work requires approval. The regular working hours are from 9 AM to 6 PM, Monday to Friday.
